# Assignment 3
Solve the 2D wave equation for eigenmodes and eigenfrequencies of membranes (square, rectangle, circle) with fixed boundaries. Discretize, compute eigenvalues, analyze frequency dependence on size, and visualize time evolution. Also, solve the steady-state diffusion equation on a circular domain.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from IPython.display import HTML

from src.config import *

----
## 3.1 Eigenmodus of drums or membranes of different shapes
Lorem ipsum odor amet, consectetuer adipiscing elit. Varius nullam vulputate lobortis porttitor in odio mus dictum. Tellus phasellus congue hac, tempus montes ullamcorper himenaeos. Arcu bibendum convallis lacus taciti rutrum ac dictumst mollis. Cras nullam est nostra netus nascetur iaculis hendrerit. Odio parturient purus maecenas consectetur porta hendrerit lobortis sodales. Maecenas quisque gravida sed nisi odio pellentesque habitasse. Cras tempus lacinia rhoncus conubia duis massa. Sem nibh dui mus consequat facilisi. Porta malesuada porttitor neque turpis molestie?

Consequat dictum curabitur integer suspendisse inceptos. Habitant mattis odio lorem nostra feugiat vitae vel habitasse. Scelerisque vehicula turpis eleifend, felis aptent venenatis ante fames. Tincidunt auctor quam ipsum velit ornare augue. Etiam habitant felis diam; ullamcorper netus aenean consectetur leo. Purus phasellus potenti tortor maecenas pulvinar. Auctor cras cursus hac tempor nam leo ad.

_by: Tycho Stam_

In [ ]:
from src.grid_discretization import (
    initialize_grid_vector,
    initialize_tridiagonal_matrix,
)
from src.eigen_solver import (
    solve_eigenvalues,
    get_frequencies_list,
    time_dependent_solution,
    normalize_eigenmodus,
)
from script.create_plot import (
    create_seaborn_heatmap,
    plot_eigenmodus,
    plot_eigenfrequency,
    plot_eigenmode_animation,
    plot_multiple_eigenmodes
)

In [ ]:
L = 4
shape = "rectangle"

v = initialize_grid_vector(L, shape=shape)
M = initialize_tridiagonal_matrix(v, L)

_, ax = plt.subplots(1, 2, figsize=(FIG_SIZE), dpi=FIG_DPI)

create_seaborn_heatmap(M.toarray(), ax[0])
ax[0].set_title(f"Matrix of size ${L**2} x {L**2}$")

create_seaborn_heatmap(v.reshape(L, L if not shape == "rectangle" else L * 2), ax[1])
ax[1].set_title(f"Vector of length ${L**2}$,\n using a {shape} as shape")

plt.show()

In [ ]:
dx = 0.01
L = 50

v = initialize_grid_vector(L, shape=shape)

In [ ]:
%%time
M = initialize_tridiagonal_matrix(vector=v, L=L, sparse=False)

M = M * dx**2
frequencies, eigenvectors = solve_eigenvalues(M, model="h")

In [ ]:
%%time
M = initialize_tridiagonal_matrix(vector=v, L=L, sparse=True)

M = M * dx**2
frequencies, eigenvectors = solve_eigenvalues(M, model="h")

In [ ]:
def plot_shape_modus(shape: str) -> None:
    """
    Plot the eigenmodes of a given shape.

    Params
    ----------
    - shape (str): The shape of the grid.
    """
    L = 55
    v = initialize_grid_vector(L, shape=shape)
    M = initialize_tridiagonal_matrix(v, L, sparse=True)
    M = M * dx**2

    frequencies, eigenvectors = solve_eigenvalues(M, num_eigen=5)
    frequencies[:PLOTS_AMOUNT], eigenvectors[:, :PLOTS_AMOUNT]

    plot_multiple_eigenmodes(PLOTS_AMOUNT, frequencies, eigenvectors, L, shape=shape)

In [ ]:
plot_shape_modus("square")

In [ ]:
plot_shape_modus("rectangle")

In [ ]:
plot_shape_modus("circle")

In [ ]:
def plot_shape_frequency(shape: str) -> None:
    """
    Plot the frequency of a given shape.

    Params
    ----------
    - shape (str): The shape of the grid.
    """
    L_list = np.linspace(1, L - 10, NUM_MODES, dtype=int)
    frequencies_list = get_frequencies_list(L, dx, L_list, shape)

    plot_eigenfrequency(L_list, frequencies_list, L, shape, save_img=False)

In [ ]:
plot_shape_frequency("square")

In [ ]:
plot_shape_frequency("rectangle")

In [ ]:
plot_shape_frequency("circle")

In [ ]:
t_list = np.linspace(0, TIME_MAX, TIME_STEPS)
L = 55 
v = initialize_grid_vector(L, L, shape="square")
M = initialize_tridiagonal_matrix(vector=v, sparse=True)
M = M * dx**2
frequencies, eigenvectors = solve_eigenvalues(M, model="h")

eigenmodus = eigenvectors[:, SELECT_MODE].reshape(L, L).real
eigenmodus = normalize_eigenmodus(eigenmodus)
frequency = frequencies[SELECT_MODE]

plot_eigenmode_animation(1.0, eigenmodus, frequency, t_list, shape, delete_img=True)

In [ ]:
HTML(f'<img src="results/wave.gif">')

-----
# 3.2 Direct methods for solving the steady state problems
Lorem ipsum odor amet, consectetuer adipiscing elit. Varius nullam vulputate lobortis porttitor in odio mus dictum. Tellus phasellus congue hac, tempus montes ullamcorper himenaeos. Arcu bibendum convallis lacus taciti rutrum ac dictumst mollis. Cras nullam est nostra netus nascetur iaculis hendrerit. Odio parturient purus maecenas consectetur porta hendrerit lobortis sodales. Maecenas quisque gravida sed nisi odio pellentesque habitasse. Cras tempus lacinia rhoncus conubia duis massa. Sem nibh dui mus consequat facilisi. Porta malesuada porttitor neque turpis molestie?

Consequat dictum curabitur integer suspendisse inceptos. Habitant mattis odio lorem nostra feugiat vitae vel habitasse. Scelerisque vehicula turpis eleifend, felis aptent venenatis ante fames. Tincidunt auctor quam ipsum velit ornare augue. Etiam habitant felis diam; ullamcorper netus aenean consectetur leo. Purus phasellus potenti tortor maecenas pulvinar. Auctor cras cursus hac tempor nam leo ad.

_by: [Name]_

In [ ]:
from src.direct_method import direct_method, plot_concentration
from src.config import DIAGONAL_VALUE, OFF_DIAGONAL_VALUE


In [ ]:
plot_concentration(L=50)

In [ ]:
# SOME CODE

-----
# 3.3 The leapfrog method - efficient time integration
In this part, we solve a simple one dimensional harmonic oscillator equation using the leapfrog method.

First, we solve the equation for position and velocity for different values of $k$ and plot the results.

Next, we investigate how the initial velocity at the half step affects the accuracy of the method by comparing it to the analytical solution and testing with different sizes of the time steps $\Delta t$.

Finally, we add an external time-dependent sinusoidal driving force and investigate what happens when the driving force is close to the original frequency of the oscillator and show the phase plot (v, x) of various frequencies. 

_by: Anezka Potesilova_

In [ ]:
from src.leapfrog import leapfrog, plot_leapfrog_various_k, plot_leapfrog_errors, plot_leapfrog_errors_start_end, plot_leapfrog_driving_force, plot_leapfrog_phase_plots

### I. Leapfrog method

We test three values of $k = 0.5, 2, 10$ and the default time step size $\Delta t = 0.01$, and we plot the position and velocity for each of them.

Total time $T$ is used to calculate the number of iterations/steps we will do, i.e. $T = \#steps \cdot \Delta t$.

In [ ]:
T = 10

def F_x(x, k):
    return -k * x

k_values = [0.5, 2, 10]
positions = []
velocities = []

for k in k_values:
    x_list, v_list = leapfrog(T, F_x, k)
    positions.append(x_list)
    velocities.append(v_list)

plot_leapfrog_various_k(positions, velocities, k_values, T, save=False)

DISCLAIMER: this part is mainly used for internal testing, to see how the curve behaves and compares to the analytical solution. It is not used to draw any major conclusions as there are better tools and measures for this.

Next, we test how the initial velocity at the half step affects the error of the method for two options of the initial velocity and two step sizes and $k = 5$. We can see that setting $v_{1/2} = 0$ introduces an error at the beginning but as the sinusoidal curve moves along, it evens out and then starts to grow again. On the other hand, for our more exact approximation, the error grows over time as expected.

Due to the sinusoidal nature, if we run the simulation for long enough, the curves will eventually match the exact solution again at least for a brief time before diverging again.

Note that we are taking $T=1000$, which means that for the default $\Delta t = 0.01$, we do a total of $100000$ steps.

In [ ]:
T = 1000
k = 5

def F_x(x, k):
    return -k * x

x_list_avg, v_list_avg = leapfrog(T, F_x, k)
x_list_zero, v_list_zero = leapfrog(T, F_x, k, initial_v="zero")

# plot_leapfrog_errors([x_list_avg, x_list_zero], [v_list_avg, v_list_zero], k, T, save=False)

Next, we test the same but for $\Delta t = 0.1$.

We can see that the error grows much faster and that indeed it starts to decrease again as the curve approaches the exact solution.

In [ ]:
T = 1000
k = 5
delta_t = 0.1

def F_x(x, k):
    return -k * x

x_list_avg, v_list_avg = leapfrog(T, F_x, k, delta_t=delta_t)
x_list_zero, v_list_zero = leapfrog(T, F_x, k, initial_v="zero", delta_t=delta_t)

# plot_leapfrog_errors([x_list_avg, x_list_zero], [v_list_avg, v_list_zero], k, T, save=False)

We can also check how the solutions diverge at the beginning of the period vs at the end by looking at $T \in [0,5]$ and $T \in [995,1000]$

In [ ]:
T = 1000
k = 5
delta_t = 0.05

def F_x(x, k):
    return -k * x

x_list_avg, v_list_avg = leapfrog(T, F_x, k, delta_t=delta_t)
x_list_zero, v_list_zero = leapfrog(T, F_x, k, initial_v="zero", delta_t=delta_t)

# plot_leapfrog_errors_start_end([x_list_avg, x_list_zero], k, T, delta_t=delta_t, save=False)

### J. External driving force

Lastly, we add the external time-dependent sinusoidal driving force, $F(t) = F_0 \sin(\omega_{drive} t)$.

We fix $F_0 = 0.5$, $\Delta t = 0.01$, $k =1$, and test for different values of $\omega_{drive}$. We take these as multiplications of the original $\omega$ to be able to investigate what happens when the frequency is close to the original frequency of the oscilator which corresponds to $\omega_{drive} \approx \omega$.

In [ ]:
T = 50
k, m = 1, 1
delta_t = 0.01
omega = (k / m)**0.5

omegas = [omega*0.3, omega, omega*1.5]
F_0 = 0.5

def F_x(x, k):
    return -k * x

positions = []
velocities = []

for omega_drive in omegas:
    def F_t(t):
        return F_0 * np.sin(omega_drive * t)

    x_list, v_list = leapfrog(T, F_x, k, F_t=F_t)
    positions.append(x_list)
    velocities.append(v_list)

plot_leapfrog_driving_force(
    positions, velocities, omegas, T, delta_t=delta_t, k=k, m=m, save=False
)

Lastly, we plot only the phase plots for 4 values of $\omega_{drive}$ to be used in the report. The previous figure wouldn't fit and would be difficult to read.

In [ ]:
T = 100
k, m = 1, 1
omega = (k / m)**0.5

omegas = [omega*0.3, omega*0.5, omega, omega*4]
F_0 = 0.5

def F_x(x, k):
    return -k * x

positions = []
velocities = []

for omega_drive in omegas:
    def F_t(t):
        return F_0 * np.sin(omega_drive * t)

    x_list, v_list = leapfrog(T, F_x, k, F_t=F_t)
    positions.append(x_list)
    velocities.append(v_list)

plot_leapfrog_phase_plots(positions, velocities, omegas, k=k, m=m, save=False)